In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Initiate Class

In [2]:
df = pd.read_csv("../data/a02_p3.csv")
df.timestamp = pd.to_datetime(df.timestamp)
# is_stay, stay_times, stay_durations = find_stays(df.x, df.y, df.z, df.timestamp, params)
# find_stays(df.x, df.y, df.z, df.timestamp, params)

In [48]:
fs = 25 #[Hz]
params = {"win_size_sec": 3,
"ecdf_diff_th": 0.01,
"var_th": 0.05,
"abrupt_filt_time_const": 10,
"abrupt_pctg_th": 0.2,
"min_stay_duration": 4,
"max_time_gap_msec": 1e3 * 5 / fs,
"max_section_gap_minutes": 7,
"max_time_gap_pctl": 60}

In [49]:
# unpack & definitions
accx = df.x.to_numpy()
accy = df.y.to_numpy()
accz = df.z.to_numpy()
timestamp = df.timestamp.to_numpy()

acc_mat = np.array([accx , accy , accz])  # stacking

MAX_HIST_BINS = int(1e4)
NUN_DIMS = 3 # {x y z}



### methods

In [59]:
# calculate average sample rate
def avg_sample_rate(time_diffs,params):
    mask = time_diffs <= np.percentile(time_diffs, params["max_time_gap_pctl"])+1 #+1 fixing numpy.percentile bug
    fs = np.mean(1e3/time_diffs[mask].astype('timedelta64[ms]').astype('float64')) #[Hz]
    return fs

In [51]:
# calc params in sample (ctor)
time_diffs = np.diff(timestamp)
fs = avg_sample_rate(time_diffs,params)
sec2smp = lambda sec: np.floor(sec*fs).astype('int32');  # util function
win_size_smp = sec2smp(params["win_size_sec"])
abrupt_filt_size = sec2smp(params["abrupt_filt_time_const"])

In [52]:
def rolling_window(arr, window):
    shape = arr.shape[:-1] + (arr.shape[-1] - window + 1, window)
    strides = arr.strides + (arr.strides[-1],)
    return np.lib.stride_tricks.as_strided(arr, shape=shape, strides=strides)

In [60]:
# optionally update var_th
def update_var_th(acc_mat,win_size_smp,MAX_HIST_BINS,params):
    acc_abs = np.sqrt(np.power(acc_mat,2).sum(axis=0))
    cc_movevar = np.var(rolling_window(acc_abs, win_size_smp), axis=1)
    hist, bin_edges = np.histogram(acc_movevar,bins=MAX_HIST_BINS)
    mvr_epdf = hist / sum(hist); # normalize to pdf
    knee_th = bin_edges[np.argwhere(mvr_epdf < params["ecdf_diff_th"])[-1]]
    if not knee_th.size:
        knee_th = params["var_th"]
    return min([params["var_th"], knee_th])

var_th = update_var_th(acc_mat,win_size_smp,MAX_HIST_BINS,params)

In [56]:
max_section_gap = np.array(params["max_section_gap_minutes"], dtype='timedelta64[m]')
section_idxs = np.append(np.array(0), np.argwhere(time_diffs > max_section_gap))
section_idxs = np.append(section_idxs, len(timestamp)-1)

In [57]:
# %% go through each section and decide isStay
# is_seperate_axis = [];  % should be realocated in other language
# is_abs_stay = [];       % should be realocated in other language
# for isect=1:length(section_idxs)-1
#     curr_section_idxs = section_idxs(isect) : section_idxs(isect+1) ;
#     curr_acc_mat = acc_mat(curr_section_idxs);
#     curr_acc_abs = sqrt(sum(curr_acc_mat.^2,2)); 
#     mvr_mat = movvar(curr_acc_mat, win_size_smp, 0, 1);
#     mvr_abs = movvar(curr_acc_abs, win_size_smp);
    
#     % check seperate axis
#     is_seperate_axis(curr_section_idxs) = all(mvr_mat < var_th/Ndims,2);
#     is_abs_stay(curr_section_idxs) = mvr_abs < var_th ;
# end
# isStay = is_seperate_axis & is_abs_stay ;
